# 05-2. 교차 검증과 그리드 서치
목표: 검증 세트가 필요한 이유를 이해하고 교차 검증에 대해 학습하기, 그리드 서치와 랜덤 서치를 이용해 최적의 성능을 내는 하이퍼파라미터 찾기


---


## 검증 세트
- 훈련 세트로 훈련한 모델의 성능을 평가할 때 테스트 세트를 사용해 자꾸 성능을 확인하면 테스트 세트에 더 잘 맞는 모델을 만들게 되는 것
- 테스트 세트로 일반화 성능을 제대로 파악하려면 테스트 세트를 최대한 덜 사용해야 함
- 이를 위해 훈련 세트에서 모델을 훈련한 뒤 검증 세트로 모델을 평가함
- 전체 세트 = 훈련 세트 60% + 검증 세트 20% + 테스트 세트 20%
- 검증 세트로 매개변수를 테스트해나가며 가장 좋은 모델을 고른 뒤,이 매개변수를 사용해 훈련+검증 세트로 모델을 훈련
- 최종적으로 테스트 세트에서 모델 최종 점수를 평가


In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

- train_test_split() 함수를 2번 적용하여 훈련 세트, 검증 세트, 테스트 세트로 나누기

In [4]:
# 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [5]:
# 훈련세트와 검증 세트 나누기
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


- 사이즈가 줄어든 훈련 세트로 모델 훈련하기

In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


- 과대적합되어 있는 상태 -> 더 좋은 매개변수를 찾아보자

## 교차 검증
- 교차 검증: 검증 세트로 평가하는 과정을 여러 번 반복한 뒤, 평균을 계산해 최종 검증 점수 얻음
- 3-폴드 교차 검증: 훈련 세트를 3부분으로 나눠 교차 검증을 수행, 훈련 세트를 몇 부분에 나누냐에 따라 k-폴드 교차 검증이라 함


In [8]:
from sklearn.model_selection import cross_validate
# 검증 세트로 나누지 않은 훈련 세트 전체를 전달
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00899935, 0.00892782, 0.00890946, 0.00911975, 0.00944662]), 'score_time': array([0.00123715, 0.00127959, 0.00138688, 0.00140834, 0.00149226]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- cross_validate()
  + 매개변수1 = 평가할 모델 객체, 매개변수2 = 훈련 세트 전체
  + fit_time(모델 훈련 시간), score_time(모델 검증 시간), test_score 키를 가진 딕셔너리 반환
  + 기본적으로 5-폴드 교차 검증을 수행하므로 5개의 숫자가 포함됨
  + cv 매개변수로 폴드 수 조정
  + 최종 교차 점수는 위 5개의 점수를 평균하여 얻음

In [9]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


- train_test_split()은 데이터를 섞어 훈련 세트와 테스트 세트로 나눔
- cross_validate()는 그런 기능 없으므로 훈련 세트를 섞으려면 분할기를 지정해야 함
- 기본적으로 회귀 모델일 경우, KFold 분할기 사용
- 분류 모델일 경우, 타킷 클래스 골고루 섞는 StratifiedKFold 분할기 사용

In [10]:
# 앞서 수행한 교차 검증은 다음 코드와 동일
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


- 훈련 세트를 섞은 후, 10-폴드 교차 검증 수행하려면
  + n_splits: 몇 폴드 교차 검증을 할지 결정

In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


- KFold 클래스도 동일한 방식으로 사용

---


- 이제 결정 트리의 매개변수 값을 바꿔가며 가장 성능이 좋은 모델을 찾아야 함
- 테스트 세트 사용하지 않고 교차 검증 사용하기


## 하이퍼파라미터 튜닝
- 하이퍼파라미터: 모델이 학습할 수 없어 사용자가 지정해야만 하는 파라미터
- 머신러닝 라이브러리에서 클래스나 메서드의 매개변수로 표현됨
- 처음엔 기본값 사용해 모델 훈련 -> 검증 세트의 점수나 교차 검증을 통해 매개변수 조금씩 수정
- 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾는 것이 아님  
-> 매개변수의 값을 동시에 바꿔가며 최적값을 찾아야함  
-> for문 대신, 사이킷런의 그리드 서치 사용
- 사이킷런의 GridSearchCV는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함, cross_validate() 호출 필요 없음  

### min_impurity_decrease 매개변수의 최적값 찾기
1. 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 생성

In [25]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

2. 탐색 대상 모델과 params 변수 전달해 그리드 서치 객체 생성


In [26]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)

3. 그리드 서치 객체 훈련하기
- 결정 트리 모델의 min_impurity_decrease 값을 바꿔가며 5번 실행함
- cv 기본값은 5,각 값마다 5폴드-교차 검증 수행 -> 5x5개의 모델 훈련
- n_jobs 매개변수로 병렬 실행에 사용할 CPU 코어수 지정, 기본값은 1, 모든 코어는 -1

In [28]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- GridSearchCV는 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트에서 자동으로 다시 모델을 훈련함
- 이 모델은 gs 객체의 best_estimator_ 속성에 저장
- 최적의 매개변수 값은 best_params_ 속성에 저장
- 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score' 키에 저장

In [29]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [30]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


- 넘파이 argmax()를 사용하면 가장 큰 값의 인덱스 추출 가능
- 그리고 그 인덱스로 params 키에 저장된 매개변수 출력

In [31]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


### 더 복잡한 매개변수 조합 탐색하기
- min_impurity_decrease: 노드를 분할하기 위한 불순도 감소 최소량 지정
- max_depth: 트리의 깊이 제한
- min_samples_split: 노드를 나누기 위한 최소 샘플 수

In [32]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

- $ 9 * 15 * 10 = 1350 $ 번의 교차 검증
- 5-폴드 교차 검증으로, $ 1350 * 5 = 6750 $ 개의 모델 훈련

In [34]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

- 최상의 매개변수 조합 확인하기

In [35]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


- 최상의 교차 검증 점수 확인하기

In [36]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치
- 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정의하기 어려울 때
- 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 때  
-> 랜덤 서치 사용
- 랜덤 서치에는 매개변수의 값의 목록이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

- scipy는 수치 계산 전용 라이브러리
- uniform(실수), randint(정수): 균등 분포에서 샘플링하는데 사용

In [39]:
from scipy.stats import uniform, randint

- 0에서 10 사이의 범위를 갖는 randint 객체에서 10개의 숫자 샘플링하기

In [40]:
rgen = randint(0, 10)
rgen.rvs(10)

array([6, 9, 4, 2, 1, 3, 6, 3, 2, 1])

- 1000개를 샘플링해서 각 숫자의 추출 횟수 확인해보기

In [41]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([112,  95, 113, 101, 104,  94,  86,  90, 105, 100]))

- uniform 클래스 사용법도 동일함

In [42]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.4932385 , 0.22890643, 0.50870863, 0.89525686, 0.78128389,
       0.41267472, 0.41153908, 0.55035494, 0.98453534, 0.300291  ])

- 랜덤 서치에 randint과 uniform 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라 할 수 있음

In [43]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}
          # min_samples_leaf: 리프 노드가 되기 위한 최소 샘플의 개수

- 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정
- 매개변수 범위에서 총 100번을 샘플링하여 교차검증을 수행하고 최적의 매개변수 조합을 찾을 수 있음
- 그리드 서치보다 교차 검증 수를 훨씬 줄이고 넓은 영역 탐색 가능

In [44]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c95bdb93800>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c95b7817fe0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c95b7815310>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c95b78158e0>},
                   random_state=42)

- 최적의 매개변수 조합 확인


In [45]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


- 최적의 교차 검증 점수 확인

In [46]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


- 이 모델을 최종 모델로 결정하고 테스트 점수 확인하기

In [47]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


## 확인 문제
마지막 RandomizedSearchCV 예제에서 DecisionTreeClassifer 클래스에 splitter='random' 매개변수를 추가하고 다시 훈련해 보자.
- splitter 매개변수의 기본값은 'best'로 각 노드에서 최선의 분할을 찾음
- 'random'이면 무작위로 분할한 다음 가장 좋은 것을 선택함
- 왜 이런 옵션이 필요한지는 다음 절에서 알 수 있음

In [49]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
model= gs.best_estimator_
print(model.score(test_input, test_target))

0.786923076923077


- 테스트 세트에서 성능이 다시 하락함 why?
